In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random
import time

In [2]:
files = ['a','b','c','d','e','f','g','h']
ranks = [1,2,3,4,5,6,7,8]
#board = pd.read_csv("chess board.csv", index_col=0)
#board = pd.read_csv("chess board kingmovetest.csv", index_col=0)
#board = pd.read_csv("chess board checktest.csv", index_col=0)
board = pd.read_csv("chess board avoidchecktest.csv", index_col=0)

In [5]:
moveDict = {'upx':('up','x'),
               'downx':('down','x'),
               'leftx':('left','x'),
               'rightx':('right','x'),
               'upxleftx':(('up','x'),('left','x'),2),
               'upxrightx':(('up','x'),('right','x'),2),
               'downxleftx':(('down','x'),('left','x'),2),
               'downxrightx':(('down','x'),('right','x'),2),
               'up1':('up',1),
               'down1':('down',1),
               'left1':('left',1),
               'right1':('right',1),
               'up1left1':(('up',1),('left',1),2),
               'up1right1':(('up',1),('right',1),2),
               'down1left1':(('down',1),('left',1),2),
               'down1right1':(('down',1),('right',1),2),
               'up2left':(('up',2),('left',1),2),
               'up2right':(('up',2),('right',1),2),
               'left2up':(('left',2),('up',1),2),
               'left2down':(('left',2),('down',1),2),
               'right2up':(('right',2),('up',1),2),
               'right2down':(('right',2),('down',1),2),
               'down2left':(('down',2),('left',1),2),
               'down2right':(('down',2),('right',1),2),
               'up2':('up',2),
                'down2':('down',2),
               'WpTakeL':(('up',1),('left',1),2),
               'WpTakeR':(('up',1),('right',1),2),
               'BpTakeL':(('down',1),('left',1),2),
               'BpTakeR':(('down',1),('right',1),2),
               'castleR': ('right',2),
               'castleL': ('left',2)}

In [6]:
board

,a,b,c,d,e,f,g,h
8,bR,bN,bB,0,bK,bB,bN,bR
7,bP,bP,bP,bP,bP,bP,bP,bP
6,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0
4,0,0,0,0,0,bQ,0,0
3,0,0,0,0,wK,0,0,0
2,wP,wP,wP,wP,wP,wP,wP,wP
1,wR,wN,wB,wQ,0,wB,wN,wR


In [7]:
board.at[3,'h']

'0'

In [8]:
#emptySquares = pd.isna(board).copy()
#emptySquares

In [9]:
#board.at[6,'a'] = board.at[7,'a']
#board.at[7,'a'] = None
#board.at[3,'a']='boo'
#board


In [10]:
def recon(rsquare):
    #print('RECON')
    #look at a square and return the piece or False
    if not rsquare:
        return False
    if rsquare == '0':
        return False
    #print('recon rsquare: ' + str(rsquare))
    file = rsquare[0]
    rank = rsquare[1]
    square = board.at[rank,file]
    #print('recon square: ' + str(square))
    return square

In [11]:
recon(('a',1))

'wR'

In [12]:
def walk(square, movement):
    #print('WALK')
    #general movement between squares
    if not square:
        return False
    file = square[0]
    rank = square[1]
    direction = movement[0]
    steps = movement[1]
    if steps == 'x':
        steps = 1
    #print(steps)
    fileCom = {"up":1, "down":-1, "left":-1, "right":1}
    global newPos
    if direction in ["left", "right"]:
        findex = files.index(file)
        nindex = findex + (steps * fileCom[direction])
        if nindex not in range(0,8):
            return False
        newFile = files[nindex]
        newPos = (newFile, rank)
    if direction in ["up", "down"]:
        newRank = (rank + (steps * fileCom[direction]))
        if newRank not in range(1,9):
            return False
        newPos = (file, newRank)
    return newPos

In [13]:
walk(('e',3), ('up', 1))

('e', 4)

In [14]:
def defaultMoves(square, castle=False, forTheKing=False):
    moves = []
    #print('DEFAULTMOVES')
    #returns default moveset for a piece
    file = square[0]
    rank = square[1]
    piece = recon(square)
    if piece == '0':
        return False
    color = piece[0]
    #print(color)
    pieceType = piece[1]
    #print(pieceType)
    if pieceType == 'P':
        #check pawn team and loc
        if color == 'w':
            moves = ['up1','WpTakeL','WpTakeR']
            if rank == 2:
                #pawn can move up 1 or 2
                moves = ['up1','up2','WpTakeL','WpTakeR']
        if color == 'b':
            moves = ['down1','BpTakeL','BpTakeR']
            if rank == 7:
                #pawn can move down 1 or 2
                moves = ['down1','down2','BpTakeL','BpTakeR']
    if pieceType == 'R':
        moves = ['upx','downx','leftx','rightx']
    if pieceType == 'N':
        moves = ['up2left','up2right','left2up','left2down','right2up','right2down','down2left','down2right']
    if pieceType == 'B':
        moves = ['upxleftx','upxrightx','downxleftx','downxrightx']
    if pieceType == 'Q':
        moves = ['upx','downx','leftx','rightx','upxleftx','upxrightx','downxleftx','downxrightx']
    if pieceType == 'K':
        if not castle:
            if forTheKing:
                moves = findKingMoves(square)
            if not forTheKing:
                moves = ['up1','down1','left1','right1','up1left1','up1right1','down1left1','down1right1']
        if castle:
            left = canCastle(color, 'l')
            right = canCastle(color,'r')
            if forTheKing:
                moves = findKingMoves(square)
            if not forTheKing:
                moves = ['up1','down1','left1','right1','up1left1','up1right1','down1left1','down1right1']
            if left and right:
                moves.extend(['castleL', 'castleR'])
            elif left:
                moves.extend(['castleL'])
            elif right:
                moves.extend(['castleR'])
    return moves

In [15]:
defaultMoves(('e',3), forTheKing=True)

NameError: name 'findKingMoves' is not defined

In [16]:
def assessSquare(square, newSquare, origin):
    #print('ASSESSSQUARE')
    check = recon(newSquare)
    if not check:
        #out of bounds
        #print('oob')
        return 'oob'
    elif check is '0':
        #then the square is empty
        #print('empty')
        return 'empty'
    else:
        #the square is either friend or foe
        #print(recon(newSquare))
        #print(recon(square))
        if recon(newSquare)[0] is not recon(origin)[0]:
            #print('foe')
            return 'foe'
        else:
            #print('friend')
            return 'friend'

In [17]:
assessSquare(('g',1),('d',8), ('b',8))

'empty'

In [18]:
def xCheck(x, move, square, imaginary=False):
    #print('XCHECK')
    origin = square
    #look through each move str name, make moves and report if possible
    if 'x' in move:
            
        #print(move + ': x')
        #need to test further moves in this direction until blocked
        blocked = False
        xSquares = []
        while not blocked:
            #print('not blocked')
            if len(x) == 3:
                #move is a two-step
                move1 = x[0]
                #print(move1)
                move2 = x[1]
                #print(move2)
                newSquare = walk(square, move1)
                #print(newSquare)
                newSquare = walk(newSquare, move2)
                #print(newSquare)
            else:
                #move is a one-step
                newSquare = walk(square, x)
                #print(newSquare)

            result = assessSquare(square, newSquare, origin)
            #print(result)
            if result == 'oob':
                blocked = True
                newSquare = False
            elif result == 'empty':
                blocked = False
                xSquares.append(newSquare)
                square = newSquare
            elif result == 'friend':
                blocked = True
                newSquare = False
            else:
                blocked = True
                xSquares.append(newSquare)
                square = newSquare
        newSquare = xSquares
        if len(newSquare) == 0:
            newSquare = False
            
    if 'x' not in move:
        #print(move + ': no x')
        #move only specified number of moves
        
        #only for castle
        if 'castle' in move:
            if 'L' in move:
                team = 'l'
            if'R' in move:
                team = 'r'
                
        #non-castle
        if len(x) == 3:
            #print(x)
            #move is a two-step
            #this is where the horses move
            move1 = x[0]
            #print(move1)
            move2 = x[1]
            #print(move2)
            newSquare = walk(square, move1)
            newSquare = walk(newSquare, move2)
        else:
            #move is a one-step
            newSquare = walk(square, x)
        result = assessSquare(square, newSquare, origin)
        if result == 'oob':
            newSquare = False
        if result == 'friend':
            newSquare = False
        if 'P' in recon(square):
            if not imaginary:
                if newSquare:
                    if newSquare[0] is not square[0]:
                        if recon(newSquare) is '0':
                            newSquare = False
                if newSquare:
                    if newSquare[0] is square[0]:
                        if recon(newSquare) is not '0':
                            newSquare = False
            if imaginary:
                if newSquare:
                    if newSquare[0] is square[0]:
                        if recon(newSquare) is not '0':
                            newSquare = False
                if newSquare:
                    if newSquare[0] is not square[0]:
                        if recon(newSquare) is '0':
                            newSquare = newSquare
    return newSquare

In [19]:
xCheck(((('down', 1), ('left', 1), 2)), 'BpTakeL', ('e',5), imaginary=True)

('d', 4)

In [20]:
def moveTester(square, castle=False, forTheKing=False, imaginary=False):
    #returns legal moves
    #print('MOVETESTER')
    legalmoves = {}
    if recon(square) == '0':
        return False
    moves = defaultMoves(square, castle=castle, forTheKing=forTheKing)
    #print(moves)
    for move in moves:
        x = moveDict[move]
        #print(move)
        #print(x)
        newSquare = xCheck(x, move, square, imaginary)
        legalmoves[move] = newSquare

    #returns a dict of moves and their resulting squares (or False)
    return legalmoves


In [21]:
board

,a,b,c,d,e,f,g,h
8,bR,bN,bB,0,bK,bB,bN,bR
7,bP,bP,bP,bP,bP,bP,bP,bP
6,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0
4,0,0,0,0,0,bQ,0,0
3,0,0,0,0,wK,0,0,0
2,wP,wP,wP,wP,wP,wP,wP,wP
1,wR,wN,wB,wQ,0,wB,wN,wR


In [22]:
moveTester(('e',5), imaginary=True)

False

In [23]:
def moveCounter(square, castle=False, forTheKing=False, imaginary=False):
    #print('MOVECOUNTER')
    #returns count of legal moves
    count = 0
    movesReturned = moveTester(square, castle=castle, forTheKing=forTheKing, imaginary=imaginary)
    #print(movesReturned)
    if movesReturned:
        for move in movesReturned.items():
            #print(move[1])
            if move[1]:
                #print(move)
                count += 1
                if len(move) > 2:
                    count += (len(move) - 1)
    return count

In [24]:
moveCounter(('e',5), imaginary=True)

0

In [29]:
def myTeam(color, castle=False, imaginary=False):
    global board
    #print('MYTEAM')
    #team is w or b (str)
    myPieces = {}
    for rank in ranks:
        for file in files:
            piece = board.at[rank,file]
            #print(piece)
            if color in piece:
                movenum = moveCounter((file,rank), castle, imaginary)
                myPieces[piece + str(file) + str(rank)] = ((file, rank), movenum)
    return myPieces

In [30]:
myTeam('w')

{'wRa1': (('a', 1), 0),
 'wNb1': (('b', 1), 2),
 'wBc1': (('c', 1), 0),
 'wQd1': (('d', 1), 1),
 'wBf1': (('f', 1), 0),
 'wNg1': (('g', 1), 2),
 'wRh1': (('h', 1), 0),
 'wPa2': (('a', 2), 2),
 'wPb2': (('b', 2), 2),
 'wPc2': (('c', 2), 2),
 'wPd2': (('d', 2), 2),
 'wPe2': (('e', 2), 1),
 'wPf2': (('f', 2), 1),
 'wPg2': (('g', 2), 2),
 'wPh2': (('h', 2), 2),
 'wKe3': (('e', 3), 5)}

In [31]:
def myMoves(teamdict):
    #print('MYMOVES')
    moves = {}
    for piece in teamdict:
        x = teamdict[piece]
        if x[1] is not 0:
            moves[piece] = x
    return moves

In [32]:
team = myTeam(board,'w')

myMoves(team)

TypeError: 'in <string>' requires string as left operand, not DataFrame

In [213]:
team

{'wRa1': (('a', 1), 1),
 'wNd1': (('d', 1), 3),
 'wBf1': (('f', 1), 1),
 'wRh1': (('h', 1), 2),
 'wPa2': (('a', 2), 2),
 'wBb2': (('b', 2), 3),
 'wPc2': (('c', 2), 2),
 'wPd2': (('d', 2), 2),
 'wQe2': (('e', 2), 5),
 'wPg3': (('g', 3), 0),
 'wNh3': (('h', 3), 3),
 'wPe4': (('e', 4), 2),
 'wPf4': (('f', 4), 0),
 'wKg4': (('g', 4), 4),
 'wPh4': (('h', 4), 0),
 'wPb5': (('b', 5), 0)}

In [244]:
def pickRandomMove(team, castle=False, specific=False):
    canmoves = []
    x = myMoves(myTeam(team))
    if len(x) == 0:
        return 'done', 'done'
    #print(x)
    while len(canmoves) == 0:
        if specific:
            pick = specific
        if not specific:
            pick = random.choice(list(x))
        if pick[1] == 'K':
            forTheKing=True
        if pick[1] is not 'K':
            forTheKing=False
        #print('pick: ' + str(pick))
        square = x[pick][0]
        moves = moveTester(square, castle, forTheKing=forTheKing)
        #print(moves)
        for y in list(moves):
            if moves[y]:
                canmoves.append(y)
    #print(canmoves)    
    movepick = random.choice(canmoves)
    return pick, movepick

In [246]:
pickRandomMove('w')


('wNb1', 'up2left')

In [115]:
def dangerDetector(square, team, forTheKing=False, imaginary=False):
    attacks = []
    enemy = False
    if team == 'w':
        enemy = 'b'
    if team == 'b':
        enemy = 'w'
    if not enemy:
        return False
    for rank in ranks:
        for file in files:
            if enemy in recon((file,rank)):
                #print('enemy: ' + str(recon((file,rank))) + str(file) + str(rank))
                moves = moveTester((file,rank), forTheKing=forTheKing, imaginary=imaginary)
                #print(moves)
                for move in moves.values():
                    if move:
                        #print('square: ' + str(square))
                        #print('move: ' + str(move))
                        if str(square) in str(move):
                            loc = (file,rank)
                            attacks.append(str(recon(loc)) + str(file) + str(rank))
    if len(attacks) == 0:
        return False
    return attacks
            

In [274]:
dangerDetector(('e',7),'b')

False

In [32]:
#def escapeCounter

In [33]:
def canCastle(team, side):
    #check for danger or teammate along king path
    dangers = 0
    if team == 'w':
        if side == 'l':
            #king is at ('e',1), rook is at ('a',1)
            #king goes to ('c',1) and left rook goes to ('d',1)
            #e1 thru c1 must not be in danger
            path = [('e',1),('d',1),('c',1)]
            
        if side == 'r':
            #king is at ('e',1), rook is at ('h',1)
            #king goes to ('g',1) and right rook goes to ('f',1)
            #e1 thru g1 must not be in danger
            path = [('e',1),('f',1),('g',1)]
            
    if team == 'b':
        if side == 'l':
            #king is at ('e',8), rook is at ('a',8)
            #king goes to ('c',8) and left rook goes to ('d',8)
            #e8 thru c8 must not be in danger
            path = [('e',8),('d',8),('c',8)]
            
        if side == 'r':
            #king is at ('e',8), rook is at ('h',8)
            #king goes to ('g',8) and rook goes to ('f',8)
            #e8 thru g8 must not be in danger
            path = [('e',8),('f',8),('g',8)]
    for spot in path:
        piece = recon(spot)
        if 'K' not in piece:
            if '0' not in piece:
                dangers += 1
            if '0' in piece:
                x = dangerDetector((spot), team)
                if x:
                    dangers += len(x)
        #print(dangers)
    if dangers == 0:
        return True
    else:
        return False
        
            

In [34]:
canCastle('w', 'r')

False

In [35]:
def castle(team, side):
    global board
    if team == 'w':
        if side == 'l':
            #king is at ('e',1), rook is at ('a',1)
            #king goes to ('c',1) and left rook goes to ('d',1)
            kingfrom = ('e',1)
            kingto = ('c',1)
            rookfrom = ('a',1)
            rookto = ('d',1)
            ('white queen side castle')
            
        if side == 'r':
            #king is at ('e',1), rook is at ('h',1)
            #king goes to ('g',1) and right rook goes to ('f',1)
            kingfrom = ('e',1)
            kingto = ('g',1)
            rookfrom = ('h',1)
            rookto = ('f',1)
            ('white king side castle')
            
    if team == 'b':
        if side == 'l':
            #king is at ('e',8), rook is at ('a',8)
            #king goes to ('c',8) and left rook goes to ('d',8)
            kingfrom = ('e',8)
            kingto = ('c',8)
            rookfrom = ('a',8)
            rookto = ('d',8)
            ('black queen side castle')
            
        if side == 'r':
            #king is at ('e',8), rook is at ('h',8)
            #king goes to ('g',8) and rook goes to ('f',8)
            kingfrom = ('e',8)
            kingto = ('g',8)
            rookfrom = ('h',8)
            rookto = ('f',8)
            print('black king side castle')
    
    ### move king ###
    board.at[kingto[1],kingto[0]] = board.at[kingfrom[1],kingfrom[0]]
    board.at[kingfrom[1],kingfrom[0]] = '0'
    #################
    ### move rook ###
    board.at[rookto[1],rookto[0]] = board.at[rookfrom[1],rookfrom[0]]
    board.at[rookfrom[1],rookfrom[0]] = '0'
    #################
    return board

In [121]:
moveDict['up1left1']

(('up', 1), ('left', 1), 2)

In [152]:
#https://stackoverflow.com/questions/627435/how-to-remove-an-element-from-a-list-by-index
#https://stackoverflow.com/questions/522563/accessing-the-index-in-for-loops
#https://stackoverflow.com/questions/11303225/how-to-remove-multiple-indexes-from-a-list-at-the-same-time

def findKingMoves(square):
    indanger = {}
    moves = ['up1','down1','left1','right1','up1left1','up1right1','down1left1','down1right1']
    indexCount = []
    for idx, move in enumerate(moves):
        y = moveDict[move]
        ### prepare the start and end positions ###
        if len(y) == 3:
            newSquare = walk(square, y[0])
            newSquare = walk(newSquare, y[1])
        else:
            newSquare = walk(square, y)
        #print(move)
        #print(square)
        #newSquare = walk(square, moveDict[move])
        #print(newSquare)
        indanger = dangerDetector(newSquare, recon(square)[0], forTheKing=False, imaginary=True)
        #print(indanger)
        if indanger:
            indexCount.append(idx)
    for index in sorted(indexCount, reverse=True):
        del moves[index]
    if len(moves) == 0:
        moves = 'checkmated'
    return moves

In [141]:
findKingMoves(('e',3))

up1
('e', 3)
('e', 4)
['bQf4']
down1
('e', 3)
('e', 2)
False
left1
('e', 3)
('d', 3)
False
right1
('e', 3)
('f', 3)
['bQf4']
up1left1
('e', 3)
('d', 4)
['bQf4']
up1right1
('e', 3)
('f', 4)
False
down1left1
('e', 3)
('d', 2)
['bQf4']
down1right1
('e', 3)
('f', 2)
['bQf4']


['down1', 'left1', 'up1right1']

In [38]:
def promote(team, square):
    global board
    options = ['Q', 'R', 'B', 'N']
    promotion = random.choice(options)
    newPiece = str(team) + promotion
    board.at[square[1],square[0]] = newPiece
    print(str(team) + 'P' + str(square[0]) + str(square[1]) + ' promoted to ' + promotion)
    #print(newPiece)

In [39]:
promote('w', ('h',8))

wPh8 promoted to Q


In [267]:
def play(steps=False, resetBoard=False):
    global turn
    global board
    global wKmoved
    global bKmoved
    global wRmoved
    global bRmoved
    wKmoved = False
    bKmoved = False
    wRmoved = {'l':False,'r':False}
    bRmoved = {'l':False,'r':False}
    if not steps:
        board = pd.read_csv("chess board.csv", index_col=0)
    if resetBoard:
        board = pd.read_csv("chess board.csv", index_col=0)
        turn = 'w'
    count = 0
    over = False
    if steps:
        step = 0
    moveMade = False
    while True: 
        ### look at team ###
        team = myTeam(turn)
        
        for piece in team.keys():
            #print(piece)
            if 'K' in piece:
                #print('found king')
                check = dangerDetector(team[piece][0], turn)
                #print(check)
                if check:
                    ### the king is currently in check and must move (or we must put the other king in check)
                    ### or the check must be otherwise blocked TODO
                    print(turn + ' king is in check')
                    piece, move = pickRandomMove(turn, specific=piece)
                    moveMade = True
                    if piece is 'done':
                        ### checkmate!
                        print(str(piece[0]) + ' king is in checkmate')
                        if turn is 'b':
                            print('white wins')
                        if turn is 'w':
                            print('black wins')
                        over = True
                if not check:
                    break
        if over:
            break
                    
        ### pick a random piece and move ###
        if not moveMade:
            piece, move = pickRandomMove(turn)
        
        if 'P' in piece:
            pawn = True
        else:
            pawn = False
        
        if piece == 'done':
            print(piece + ' after ' + str(count) + ' turns')
            break
        
        
        ### non castle moves ###
        if 'castle' not in move:
            y = moveDict[move]
            ### prepare the start and end positions ###
            square = team[piece][0]
            start = square
            if len(y) == 3:
                newSpace = walk(square, y[0])
                newSpace = walk(newSpace, y[1])
            else:
                newSpace = walk(square, y)
            end = newSpace

            ### detects if 'take' needs to be said instead of 'to' ###
            if assessSquare(start, end, start) == 'foe':
                totake = ' takes '
                endpiece = str(recon(end))
            else:
                totake = ' to '
                endpiece = ''
                
            ### this chunk keeps track of stuff necessary for castling ###
            if recon(start) == 'wK':
                wKmoved = True
            if recon(start) == 'bK':
                bKmoved = True
            if recon(start) == 'wR':
                if start == ('a',1):
                    wRmoved['l'] = True
                if start == ('h',1):
                    wRmoved['r'] = True
            if recon(start) == 'bR':
                if start == ('a',8):
                    bRmoved['l'] = True
                if start == ('h',8):
                    bRmoved['r'] = True
                    
            ### this moves a piece ###
            #print(recon(start))
            #print(start)
            #print(end)
            print(str(recon(start)) + str(start[0]) + str(start[1]) + str(totake) + str(endpiece) + str(end[0]) + str(end[1]))

            board.at[newSpace[1], newSpace[0]] = board.at[square[1], square[0]]
            board.at[square[1], square[0]] = '0'
            
            if pawn:
                if turn == 'w':
                    if newSpace[1] == 8:
                        promote(turn, newSpace)
                if turn == 'b':
                    if newSpace[1] == 1:
                        promote(turn, newSpace)
        
        ### castle ###
        
        if 'castle' in move:
            if 'L' in move:
                side = 'l'
            if 'R' in move:
                side = 'r'
            castle(turn, side)
            if team is 'w':
                wRmoved[side] = True
                wKmoved = True
            if team is 'b':
                BRmoved[side] = True
                BKmoved = True
                
        ### keeps track of whose turn it is ###
        q = 0
        if turn == 'b':
            turn = 'w'
            q = 1
        if turn == 'w' and q == 0:
            turn = 'b'
        #######################################
        
        #print(board)
        count += 1
        if steps:
            step += 1
            if step == steps:
                return 'paused after ' +  str(step) + ' steps'
        #time.sleep(0.5)
        

In [33]:
play(10000, resetBoard=True)

NameError: name 'play' is not defined

In [270]:
board

,a,b,c,d,e,f,g,h
8,bR,0,bB,0,0,0,0,bR
7,0,bP,0,bQ,bK,0,bP,0
6,bP,0,0,bB,0,bP,0,0
5,wP,0,bP,wP,bP,0,0,bP
4,0,wP,wP,wP,0,wN,wP,wP
3,wN,0,0,0,0,0,wP,wB
2,0,wB,0,0,wQ,0,0,0
1,0,0,wR,0,0,wK,0,wR


In [271]:
team

{'wRa1': (('a', 1), 1),
 'wNd1': (('d', 1), 3),
 'wBf1': (('f', 1), 1),
 'wRh1': (('h', 1), 2),
 'wPa2': (('a', 2), 2),
 'wBb2': (('b', 2), 3),
 'wPc2': (('c', 2), 2),
 'wPd2': (('d', 2), 2),
 'wQe2': (('e', 2), 5),
 'wPg3': (('g', 3), 0),
 'wNh3': (('h', 3), 3),
 'wPe4': (('e', 4), 2),
 'wPf4': (('f', 4), 0),
 'wKg4': (('g', 4), 4),
 'wPh4': (('h', 4), 0),
 'wPb5': (('b', 5), 0)}

In [ ]:
#track how often things occur on each square